In [ ]:
! pip install dabl
import dabl

In [ ]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import math
import time
import datetime
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc, balanced_accuracy_score
from sklearn.model_selection import GroupKFold,train_test_split,cross_validate
from sklearn import tree #decision Tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.model_selection import KFold


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

#random

In [ ]:
import os
print(os.getcwd())
fullpath = os.path.abspath(r"C:\Users\tansh\Downloads\Telegram Desktop\cleaned_final (2).csv") #change path
start_time = time.time()
df = pd.read_csv(fullpath)
log_time = (time.time() - start_time)
print(log_time)


In [ ]:
selected_df = df.drop(columns = ["CONS_ZONE_SPD_LIM"],axis = 1)
#selected_df = selected_df.drop(df.columns[0],axis = 1) #drop axis
selected_df = selected_df[selected_df["MAX_SEVERITY_LEVEL"]!= 9]
print(selected_df.info())
X_train = selected_df.drop('MAX_SEVERITY_LEVEL',axis=1)
y_train = selected_df.MAX_SEVERITY_LEVEL

# Count NaN values
# Find columns with more than one null value
cols_with_nulls = X_train.columns[X_train.isnull().sum() > 1]

# Sum the number of null values for those columns
null_counts = X_train[cols_with_nulls].isnull().sum()
null_counts

X_train.dtypes


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import chi2_contingency
from prince import FAMD

# Split the data into training and testing sets
X = selected_df.drop("MAX_SEVERITY_LEVEL",axis = 1)
y = selected_df.MAX_SEVERITY_LEVEL
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit a Random Forest model to assess feature importance
rf = RandomForestClassifier()
rf.fit(X_train_class, y_train_class)

# Get feature importances from the trained Random Forest model
feature_importances = pd.Series(rf.feature_importances_, index=X_train_class.columns)

# Sort the features by importance in descending order
feature_importances.sort_values(ascending=False, inplace=True)


# Select the top 50 features based on feature importances
top_features = feature_importances.head(50).index.tolist()
'''
# Apply FAMD for dimensionality reduction
#famd = FAMD(n_components=2)
#famd.fit(X)  # Fit FAMD to the data
#X_famd = famd.fit_transform(X)

# Train a Decision Tree model on the reduced feature set
X_train_famd, X_test_famd, y_train, y_test = train_test_split(X[top_features], y, test_size=0.2, random_state=42)
dt = DecisionTreeClassifier()
dt.fit(X_train_famd, y_train)

# Predict using the trained Decision Tree model
y_pred = dt.predict(X_test_famd)
print(X_test_famd)
print(y_pred)
# Evaluate the performance of the Decision Tree model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
'''
x_train,x_test,y_train,y_test = train_test_split(X[top_features],y,test_size = 0.3,random_state=42)

In [ ]:
pd.set_option('display.max_rows', None)
feature_importances.iloc[:] = feature_importances.iloc[:].round(4)

print(feature_importances)

#feature_importances[]=feature_importances[feature_importances[]]
X_train.info()
plt.plot(feature_importances)

In [ ]:
# LightGBM (w RandomForest input) (BALANCED)

# Fit a Random Forest model to assess feature importance
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

# Get feature importances from the trained Random Forest model
feature_importances = pd.Series(rf.feature_importances_, index=x_train.columns)

# Sort the features by importance in descending order
feature_importances.sort_values(ascending=False, inplace=True)

best_train= 0
best_acc = 0
best_no_of_samples = 0
top_features = feature_importances.head(50).index.tolist()
x_train_selected = x_train[top_features]

# LightGBM training
params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'num_class': 4, 
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'bagging_freq': 5,
    'verbose': 0
}

train_data = lgb.Dataset(x_train_selected, label=y_train)
test_data = lgb.Dataset(x_test[top_features], label=y_test)
model = lgb.train(params, train_data, valid_sets=test_data, num_boost_round=1000, early_stopping_rounds=100)

y_pred = model.predict(x_test[top_features])
y_pred_class = np.argmax(y_pred, axis=1)

accuracy = balanced_accuracy_score(y_pred_class,y_test)
precision = precision_score(y_test, y_pred_class, average='weighted')
recall = recall_score(y_test, y_pred_class, average='weighted')
f1_score = f1_score(y_test, y_pred_class, average='weighted')
confusion_mat = confusion_matrix(y_test, y_pred_class)

print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: ",(f1_score))
print("Confusion Matrix:\n", confusion_mat)
print('best accuracy: ', accuracy)

In [ ]:
#Decision Tree

from scipy.stats import randint as sp_randint 
from sklearn.model_selection import RandomizedSearchCV

tree = DecisionTreeClassifier()

# define the hyperparameters to tune
param_dist = {"max_depth": [5, 7, 9, 11, 13, 15, -1],
              'criterion': ['gini', 'entropy'],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4],
                'max_features': ['auto', 'sqrt', 'log2', None],
                "class_weight": ['balanced']}

# define the randomized search with 5-fold cross-validation
random_search = RandomizedSearchCV(tree, param_distributions=param_dist, n_iter=100, cv=5, random_state=42, n_jobs=-1)

# fit the randomized search on the training data
random_search.fit(x_train, y_train)

# evaluate the best model on the test data
best_model = random_search.best_estimator_
test_acc = best_model.score(x_test, y_test)
print('Test accuracy:', test_acc)

# Compute predictions using the best_rf model
predictions = random_search.predict(x_test)



In [ ]:
# Compute precision, recall, and F1-score
from sklearn import metrics
#accuracy = metrics.balanced_accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
#f1SCORE = f1_score(y_test, predictions, average='weighted')

# Compute confusion matrix
confusion_mat = confusion_matrix(y_test, predictions)

# Print the results
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
#print("F1-score: {:.4f}".format(f1SCORE))
print("Confusion Matrix:\n", confusion_mat)


# print the best hyperparameters
print('Best hyperparameters:', random_search.best_params_)

In [ ]:
# Random Forest
from scipy.stats import randint as sp_randint 
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestClassifier(random_state=0)

# define the hyperparameters to tune
param_dist = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2', None],  # Number of features to consider for the best split
    'class_weight': ['balanced']  # Class weight option
}


# define the randomized search with 5-fold cross-validation
random_search = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=100, cv=5, random_state=42, n_jobs=-1)

# fit the randomized search on the training data
random_search.fit(x_train, y_train)

# evaluate the best model on the test data
best_model = random_search.best_estimator_
test_acc = best_model.score(x_test, y_test)
print('Test accuracy:', test_acc)

# Compute predictions using the best_rf model
predictions = random_search.predict(x_test)



In [ ]:
# Compute precision, recall, and F1-score

balanced = metrics.balanced_accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
#f1_score = f1_score(y_test, predictions, average='weighted')

# Compute confusion matrix
confusion_mat = confusion_matrix(y_test, predictions)

# Print the results
print("Balanced Accuracy: {:.4f}".format(balanced))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
#print("F1-score: {:.4f}".format(f1_score))
print("Confusion Matrix:\n", confusion_mat)

# print the best hyperparameters
print('Best hyperparameters:', random_search.best_params_)

In [ ]:
# Tensorflow (Neural Network)

import tensorflow as tf
import keras
from sklearn.metrics import balanced_accuracy_score

# Random Forest for feature selection
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

feature_importances = pd.Series(rf.feature_importances_, index=x_train.columns)
feature_importances.sort_values(ascending=False, inplace=True)

top_features = feature_importances.head(50).index.tolist()
x_train_selected = x_train[top_features]

# This custom class is adapted by user Aaron Keesing on Stack Overflow: https://stackoverflow.com/questions/59339531/balanced-accuracy-score-in-tensorflow
class BalancedSparseCategoricalAccuracy(keras.metrics.SparseCategoricalAccuracy):
    def __init__(self, name='balanced_sparse_categorical_accuracy', dtype=None):
        super().__init__(name, dtype=dtype)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_flat = y_true
        if y_true.shape.ndims == y_pred.shape.ndims:
            y_flat = tf.squeeze(y_flat, axis=[-1])
        y_true_int = tf.cast(y_flat, tf.int32)

        cls_counts = tf.math.bincount(y_true_int)
        cls_counts = tf.math.reciprocal_no_nan(tf.cast(cls_counts, self.dtype))
        weight = tf.gather(cls_counts, y_true_int)
        return super().update_state(y_true, y_pred, sample_weight=weight)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train_selected.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=[BalancedSparseCategoricalAccuracy(name='balanced_sparse_categorical_accuracy')])

model.fit(x_train_selected, y_train, epochs=100, batch_size=32, validation_split=0.1)

test_loss, test_accuracy = model.evaluate(x_test[top_features], y_test)
y_pred = model.predict(x_test[top_features])
y_pred_class = np.argmax(y_pred, axis=1)
precision = precision_score(y_test, y_pred_class, average='weighted')
recall = recall_score(y_test, y_pred_class, average='weighted')
f1 = f1_score(y_test, y_pred_class, average='weighted')

print("Test Accuracy: ", test_accuracy)
print("Best Precision: ", precision)
print("Best Recall: ", recall)
print("Best F1: ", f1)